<a href="https://colab.research.google.com/github/bhoomika297/Langchain-projects/blob/main/Langchain_Basics_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install langchain langchain_groq langchain_core langgraph>0.2.27

In [42]:
import warnings
warnings.filterwarnings('ignore')

In [21]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

··········


In [6]:
os.environ["GROQ_API_KEY"] = getpass.getpass()

from langchain_groq import ChatGroq

model = ChatGroq(model="llama3-8b-8192")

··········


In [43]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi,I am Bhoomika?")])



AIMessage(content='Nice to meet you, Bhoomika!', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 17, 'total_tokens': 27, 'completion_time': 0.008333333, 'prompt_time': 0.017222786, 'queue_time': 0.007704704, 'total_time': 0.025556119}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'stop', 'logprobs': None}, id='run-9ed4500b-7ff0-4452-8cd3-ad3d14a9ae7b-0', usage_metadata={'input_tokens': 17, 'output_tokens': 10, 'total_tokens': 27})

### This is the response given by model for the below prompt

content="I'm happy to try! However, I'm a large language model, I don't have the ability to know your personal information, including your name. Each time you interact with me, it's a new conversation, and I don't retain any information about you or your previous conversations. So, I won't be able to recall your name. Would you like to introduce yourself? I'd be happy to chat with you!"

In [44]:
model.invoke([HumanMessage(content="Hey! can you tell me my name?")])

AIMessage(content="I'd be happy to try! However, I'm a large language model, I don't have the ability to know your personal information or remember your name. Each time you interact with me, it's a new conversation and I don't retain any information from previous chats.\n\nIf you'd like to tell me your name, I'd be happy to learn it and use it in our conversation!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 80, 'prompt_tokens': 19, 'total_tokens': 99, 'completion_time': 0.066666667, 'prompt_time': 0.012501312, 'queue_time': 0.017198108, 'total_time': 0.079167979}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_af05557ca2', 'finish_reason': 'stop', 'logprobs': None}, id='run-d84d28bb-b26f-4e80-b8d3-671339777399-0', usage_metadata={'input_tokens': 19, 'output_tokens': 80, 'total_tokens': 99})

Lets! try to solve this perticular problem

In [45]:
from langchain_core.messages import AIMessage

model.invoke([
    HumanMessage(content="Hi, my name is Bhoomika"),
    AIMessage(content= "Hello Bhoomika! It's nice to meet you. Is there something I can help you with or would you like to chat?"),
    HumanMessage(content="What's my name?"),
])

#content='Your name is Bhoomika!

AIMessage(content='Your name is Bhoomika!', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 60, 'total_tokens': 68, 'completion_time': 0.006666667, 'prompt_time': 0.007973398, 'queue_time': 0.006031442, 'total_time': 0.014640065}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'stop', 'logprobs': None}, id='run-1f317af8-baa6-4855-8a06-11217146c516-0', usage_metadata={'input_tokens': 60, 'output_tokens': 8, 'total_tokens': 68})

Wrapping our chat model in a minimal langgraph application allows us to autometically persist the chat history

In [46]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState):
  response = model.invoke(state["messages"])
  return {"messages":response}

workflow.add_edge(START,"model")
workflow.add_node("model",call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)



In [47]:
config = {"configurable":{"thread_id":"1234"}}

In [48]:
query = "Hi, I am Bhoomika Agrahari."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages":input_messages},config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Nice to meet you, Bhoomika Agrahari! It's great to have you here. Is there something I can help you with or would you like to chat about something in particular?


In [49]:
query = "Hey! what is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages":input_messages},config)
output['messages'][-1].pretty_print()

================================== Ai Message ==================================

That's an easy one! Your name is Bhoomika Agrahari!


IMPLEMENTING THROUGH PROMPT TEMPLATE

In [50]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [51]:
prompt  = ChatPromptTemplate([
    ('system',
     'You are an expert humanoid chatbot, answer the question in the most realistic manner.',),
    MessagesPlaceholder(variable_name='messages')
])

In [52]:
workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState):
  chain = prompt | model
  response = chain.invoke(state)
  return {'messages':response}

workflow.add_edge(START,'model')
workflow.add_node('model',call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [53]:
config = {"configurable": {'thread_id':'1234'}}

query = "Hi, this is Nobita"

input_messages = HumanMessage(query)
output = app.invoke({"messages":input_messages},config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Hello there Nobita! *waves* It's great to meet you! I've heard a lot about your amazing adventures with Doraemon. What's new with you and your robotic cat friend?


In [54]:
query = "What is my name?"

input_messages = HumanMessage(query)
output = app.invoke({"messages":input_messages},config)
output['messages'][-1].pretty_print()

================================== Ai Message ==================================

Your name is Nobita Nobi, isn't it?


In [55]:

query = "Yes! please tell me a joke."


input_messages = [HumanMessage(query)]
for chunk, metadata in app.stream(
    {"messages": input_messages},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="")

Here's one: Why did Doraemon bring a time machine to Nobita's house?

(Wait for it...)

Because he wanted to take Nobita back to the time when he was still a good student! (hahaha)